# Q&A json

In [2]:
import json
from elasticsearch import Elasticsearch
from tqdm import tqdm
from openai import OpenAI
import os,sys

In [76]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# add manual dog breed related informations
# Function to load JSON file
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# Load the JSON file
questions_data = load_json("dog_json.json")
for docx in questions_data:
    documents.append(docx)

In [79]:
es = Elasticsearch("http://localhost:9200")
#create index schema

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

#initialise indice
# This Elasticsearch Python client method creates an index with the specified settings
index_name = "streamlit_w_dogs"
response = es.indices.create(index=index_name, body=index_settings)

In [80]:
# load Q&A dictionary into the indice
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████| 953/953 [00:07<00:00, 129.68it/s]


In [81]:
def check_similarities(user_question, index_name = "llm-course-questions" , max_results = 5):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": user_question,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "llm-course-questions"
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    documents = [hit["_source"] for hit in response['hits']['hits']]
    return documents
    
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

def rag(query,index_name):
    search_results = check_similarities(query,index_name)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [83]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

rag("can i be gay?","streamlit_w_dogs")

" I'm sorry, but based on the provided context, it doesn't specifically answer your question about being gay. However, please note that sexual orientation is not a choice and everyone has their own unique identity. For more information regarding this topic, you may need to consult with a professional or relevant authority in human sciences."

In [84]:
rag("what is a prague shepherd","streamlit_w_dogs")

' The Prague Shepherds might be the best suited dog for the Czech weather which can be unforgiving.\n\n(Note: There is no detailed information provided on what specifically defines a Prague Shepherd, but based on the given CONTEXT, it suggests that they are suitable for the Czech climate.)'

In [85]:
rag("what is a prague shepherd","llm-course-questions")

' I\'m sorry, but there is no information available in the provided context about "a Prague Shepherd." However, if you are referring to a breed of dog or something related to Prague, please provide more details so that I can assist you better. If you have any specific questions regarding dogs from the Czech Republic or anything associated with Prague, feel free to ask!'